In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

from nltk.tokenize import word_tokenize

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import find_missing_wsc_words_in_corpus_vocab

from src.wsc_parser import generate_df_from_json, generate_json, generate_full_sentences

In [3]:
df = generate_df_from_json()

In [4]:
import json
with open('data/processed/WSC_associative_label.json', 'r') as fp:
    english_associative_json = json.load(fp)

In [5]:
df['is_associative'] = False
for item in english_associative_json:
    if item['is_associative'] == 1:
        df.loc[item['index'],'is_associative'] = True

In [6]:
df[df['is_associative']].count().values[0]

37

In [7]:
import json
with open('data/processed/WSC_switched_label.json', 'r') as fp:
    english_switched_json = json.load(fp)

In [8]:
df['is_switchable'] = False
for item in english_switched_json:
    if item['is_switchable'] == 1:
        df.loc[item['index'],'is_switchable'] = True

In [9]:
df[df['is_switchable']].count().values[0]

131

In [10]:
import re
for item in english_switched_json:
    df.loc[item['index'],'substitution_a'] = item['answer0']
    df.loc[item['index'],'substitution_b'] = item['answer1']
    df.loc[item['index'],'correct_answer'] = 'A' if item['answer0'] == item['correct_answer'] else 'B'
    pronoun = re.findall(r'\[(.*?)\]', item['sentence_switched'])
    df.loc[item['index'],'pronoun'] = pronoun
    df.loc[item['index'],'snippet'] = '[' + pronoun[0] + ']'
    df.loc[item['index'],'switched'] = item['sentence_switched']

In [11]:
df['correct_switched'], df['incorrect_switched'] = zip(*df.apply(generate_full_sentences, axis=1))

In [12]:
def capitalize_words(sentence):
    words = []
    # changed language of tokenizer below
    for word in word_tokenize(sentence):
        if word.capitalize() in capitalized_words or (len(words) >= 1 and words[-1][-1] in ['.', '!', '?']):
            word = word.capitalize()
        if len(words) >= 1 and words[-1] == '``':
            words[-1] = '"' + word
        elif word in ['.', ',', '!', '?', ';', "''"]:
            if word == "''":
                word = '"'
            words[-1] += word
        else:
            words += [word]
    sentence = ' '.join(words).strip()
    sentence = sentence.replace('" eu primeiro! "', '"Eu primeiro"!')
    sentence = sentence.replace('" Eu primeiro! "', '"Eu primeiro"!')
    sentence = sentence.replace('tv', 'TV')
    sentence = sentence.replace('tv.', 'TV.')
    
    return sentence

In [13]:
with open('data/processed/portuguese_capitalized_words.txt', 'r') as capitalized_words_file:
    capitalized_words = [line.strip() for line in capitalized_words_file.readlines()]

In [14]:
df['switched'] = df['switched'].apply(capitalize_words)
df['correct_sentence'] = df['correct_sentence'].apply(capitalize_words)
df['incorrect_sentence'] = df['incorrect_sentence'].apply(capitalize_words)
df['correct_switched'] = df['correct_switched'].apply(capitalize_words)
df['incorrect_switched'] = df['incorrect_switched'].apply(capitalize_words)

In [15]:
df['manually_fixed_correct_sentence'], df['manually_fixed_incorrect_sentence'], \
    df['manually_fixed_correct_switched'], df['manually_fixed_incorrect_switched'] = \
    df['correct_sentence'], df['incorrect_sentence'], df['correct_switched'], df['incorrect_switched']

In [16]:
df['translated'] = True

In [17]:
generate_json(df)